In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

from const import DATA_DIR

stockpiles_path = DATA_DIR / 'nuclear-weapons' / 'nuclear-warhead-stockpiles-lines' / 'nuclear-warhead-stockpiles-lines.csv'
df_stockpiles = pd.read_csv(stockpiles_path)

pio.templates.default = "plotly_dark"

major_powers_colormap = {
    'United States': '#56CCF2',
    'Russia':        '#FF6B6B',
    'China':         '#FFE66D',
    'France':        '#BB6BD9',
    'United Kingdom':'#95A5A6',
    'World':         '#F0F3F4' 
}
major_powers = list(major_powers_colormap.keys())

stockpiles_filtered = df_stockpiles.copy()

# Group non-major powers as "Other"
stockpiles_filtered.loc[~stockpiles_filtered['Entity'].isin(major_powers), 'Entity'] = 'Other'
stockpiles_filtered = stockpiles_filtered.groupby(['Year', 'Entity'], as_index=False)['Number of nuclear warheads'].sum()

# Sorting
max_warheads = stockpiles_filtered.groupby('Entity')['Number of nuclear warheads'].max()
sorted_entities = max_warheads.sort_values(ascending=False).index.tolist()

# plot
fig1 = px.line(stockpiles_filtered, 
            x='Year', 
            y='Number of nuclear warheads', 
            width=800,
            height=400,
            color='Entity',
            labels={'Entity': "Country"},
            title='<b>The Rise & Fall of Nuclear Arsenals</b><br><sub>Tracking the accumulation of warheads across the Cold War and beyond</sub>',
            template='plotly_dark',
            category_orders={'Entity': sorted_entities},
            color_discrete_map=major_powers_colormap)

fig1.update_layout(hovermode="x unified", xaxis_title=None)
fig1.update_traces(hovertemplate='%{y}')
fig1.show()

In [2]:
from const import VISUALIZATIONS_DIR

fig1.write_html(VISUALIZATIONS_DIR / "rise-and-fall-of-nuclear-arsenals.html")

In [1]:
import pandas as pd
import plotly.graph_objects as go
from const import DATA_DIR

# 1. Caricamento e Preparazione dei Dati
def load_and_process_data(csv_path):
    df = pd.read_csv(csv_path)

    main_countries = ['United States', 'Russia']
    other_countries = ['United Kingdom', 'France', 'China', 'India', 'Pakistan', 'Israel', 'North Korea']
    
    target_countries = main_countries + other_countries
    df = df[df['Entity'].isin(target_countries)]
    
    df_pivot = df.pivot(index='Year', columns='Entity', values='Number of nuclear warheads')
    
    all_years = range(df['Year'].min(), df['Year'].max() + 1)
    df_pivot = df_pivot.reindex(all_years)
    df_pivot = df_pivot.ffill().fillna(0)
    
    df_pivot['Others'] = df_pivot[other_countries].sum(axis=1)
    df_pivot['World'] = df_pivot[main_countries + other_countries].sum(axis=1)
    
    fixed_order = ['World', 'United States', 'Russia', 'Others']
    df_clean = df_pivot[fixed_order].reset_index()
    
    return df_clean, fixed_order

# 2. Creazione del Grafico Interattivo Migliorato
def create_interactive_plot(df, countries):
    colors = {
        'World': '#ffffff',
        'United States': '#56CCF2',
        'Russia': '#f87171',
        'Others': '#9ca3af'
    }
    
    max_y = df[countries].max().max()
    
    # Eventi con posizioni ottimizzate e dimensioni box
    events = [
        # {
        #     'year': 1945, 
        #     'text': "Trinity Test / Hiroshima & Nagasaki", 
        #     'val_col': 'United States', 
        #     'text_x': 1958, 
        #     'text_y': 6000,
        #     'box_w': 18,
        #     'box_h': 3000,
        #     'text_offset': -150,
        # },
        {
            'year': 1986, 
            'text': "Chornobyl Disaster", 
            'val_col': 'World', 
            'text_x': 1997, 
            'text_y': 65000,
            'box_w': 12,
            'box_h': 6000,
            'text_offset': -300,
        }, 
        {
            'year': 1991, 
            'text': "USSR Collapse", 
            'val_col': 'United States', 
            'text_x': 2000, 
            'text_y': 38000,
            'box_w': 10,
            'box_h': 6000,
            'text_offset': -300,
        }, 
    ]
    
    for event in events:
        event_row = df[df['Year'] == event['year']]
        if not event_row.empty:
            event['y_val'] = float(event_row[event['val_col']].iloc[0])
        else:
            event['y_val'] = 0.0

    fig = go.Figure()

    # Aggiungi tracce per i paesi
    for country in countries:
        fig.add_trace(go.Scatter(
            x=[df['Year'].iloc[0]],
            y=[df[country].iloc[0]],
            mode='lines',
            name=country,
            legendgroup=country,
            line=dict(color=colors.get(country, '#ffffff'), width=4 if country == 'World' else 2.5),
            connectgaps=True,
            hovertemplate=f"<b>{country}</b>: %{{y:,.0f}}<extra></extra>"
        ))

    # Aggiungi tracce per gli eventi (4 tracce per evento: linea, marker, rettangolo sfondo, testo)
    for event in events:
        # 1. Linea connettore (tratteggiata)
        fig.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines',
            legendgroup=event['val_col'],
            showlegend=False,
            line=dict(color='#fbbf24', width=1.5, dash='dot'),
            hoverinfo='skip'
        ))
        
        # 2. Marker sul punto della linea
        fig.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='markers',
            legendgroup=event['val_col'],
            showlegend=False,
            marker=dict(size=10, color='#fbbf24', symbol='circle',
                       line=dict(color='#0f0f0f', width=2)),
            hoverinfo='skip'
        ))
        
        # 3. Rettangolo sfondo (fill='toself')
        fig.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='lines',
            fill='toself',
            fillcolor='rgba(15,15,15,0.95)',
            line=dict(color='#fbbf24', width=1.5),
            legendgroup=event['val_col'],
            showlegend=False,
            hoverinfo='skip'
        ))
        
        # 4. Testo dell'evento
        fig.add_trace(go.Scatter(
            x=[],
            y=[],
            mode='text',
            legendgroup=event['val_col'],
            showlegend=False,
            text=[''],
            textposition='middle center',
            textfont=dict(size=10, color='#fbbf24', family='Inter, Arial, sans-serif'),
            hoverinfo='skip'
        ))

    frames = []
    years = df['Year'].tolist()
    first_year = years[0]
    
    for k, year in enumerate(years):
        frame_data = []
        
        # Dati per le linee dei paesi
        for country in countries:
            frame_data.append(go.Scatter(
                x=df['Year'].iloc[:k+1],
                y=df[country].iloc[:k+1]
            ))
        
        # Dati per gli eventi (4 tracce per evento)
        for event in events:
            if year >= event['year']:
                # Calcola coordinate rettangolo
                cx, cy = event['text_x'], event['text_y']
                hw, hh = event['box_w'] / 2, event['box_h'] / 2
                rect_x = [cx - hw, cx + hw, cx + hw, cx - hw, cx - hw]
                rect_y = [cy - hh, cy - hh, cy + hh, cy + hh, cy - hh]
                
                # Linea connettore
                frame_data.append(go.Scatter(
                    x=[event['year'], event['text_x']],
                    y=[event['y_val'], event['text_y']]
                ))
                # Marker
                frame_data.append(go.Scatter(
                    x=[event['year']],
                    y=[event['y_val']]
                ))
                # Rettangolo sfondo
                frame_data.append(go.Scatter(
                    x=rect_x,
                    y=rect_y
                ))
                # Testo
                frame_data.append(go.Scatter(
                    x=[event['text_x']],
                    y=[event['text_y'] + event.get('text_offset', 0)],
                    text=[f"<b>{event['text']}</b>"]
                ))
            else:
                # Evento non ancora visibile - tracce vuote
                frame_data.append(go.Scatter(x=[], y=[]))
                frame_data.append(go.Scatter(x=[], y=[]))
                frame_data.append(go.Scatter(x=[], y=[]))
                frame_data.append(go.Scatter(x=[], y=[], text=[]))
        
        # Annotazione per l'anno corrente
        frame_annotations = [
            dict(
                x=0.98,
                y=0.98,
                xref='paper',
                yref='paper',
                text=f"<b>{year}</b>",
                showarrow=False,
                font=dict(size=60, color='rgba(255,255,255,0.12)')
            )
        ]

        frames.append(go.Frame(
            data=frame_data,
            layout=dict(annotations=frame_annotations),
            name=str(year)
        ))

    fig.frames = frames

    initial_annotations = [
        dict(
            x=0.98,
            y=0.98,
            xref='paper',
            yref='paper',
            text=f"<b>{first_year}</b>",
            showarrow=False,
            font=dict(size=60, color='rgba(255,255,255,0.12)')
        )
    ]

    fig.update_layout(
        width=800,
        height=400,
        title=dict(
            text='<b>The Rise & Fall of Nuclear Arsenals</b><br><sub>Tracking the accumulation of warheads across the Cold War and beyond</sub>',
            x=0.0,
            font=dict(size=16)
        ),
        template="plotly_dark",
        paper_bgcolor='#0f0f0f',
        plot_bgcolor='#1a1a1a',
        font=dict(family="Inter, Arial, sans-serif", color='#e5e5e5'),
        annotations=initial_annotations,
        xaxis=dict(
            showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.05)',
            range=[1945, 2025],
            fixedrange=True,
            zeroline=False,
            title=None
        ),
        yaxis=dict(
            title=dict(text="Number of nuclear warheads"),
            gridcolor='rgba(255, 255, 255, 0.05)',
            range=[0, max_y * 1.15],
            tickformat=','
        ),
        hovermode="x unified",
        hoverlabel=dict(
            bgcolor='rgba(30,30,30,0.95)',
            font=dict(size=12)
        ),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor="rgba(0,0,0,0.5)",
            tracegroupgap=0,
            itemsizing='constant',
            font=dict(size=11),
            itemwidth=30
        ),
        margin=dict(l=60, r=50, t=80, b=120),
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            x=0.0,
            y=-0.25,
            xanchor='left',
            yanchor='top',
            direction="left",
            pad=dict(r=10, t=10),
            bgcolor='#262626',
            bordercolor='#404040',
            borderwidth=1,
            font=dict(color='#ffffff', size=14),
            buttons=[
                dict(label="⏵",
                     method="animate",
                     args=[None, dict(frame=dict(duration=50, redraw=True),
                                      fromcurrent=True,
                                      transition=dict(duration=0))]),
                dict(label="⏸",
                     method="animate",
                     args=[[None], dict(frame=dict(duration=0, redraw=False),
                                        mode="immediate",
                                        transition=dict(duration=0))]),
                dict(label="⟲",
                     method="animate",
                     args=[[str(first_year)], dict(frame=dict(duration=0, redraw=True),
                                                    mode="immediate",
                                                    transition=dict(duration=0))])
            ]
        )],
        sliders=[dict(
            steps=[dict(method='animate',
                        args=[[str(y)], dict(mode='immediate',
                                           frame=dict(duration=0, redraw=True),
                                           transition=dict(duration=0))],
                        label=str(y)) for y in years],
            active=0,
            x=0.15,
            len=0.85,
            y=-0.25,
            pad=dict(t=20, b=20),
            currentvalue=dict(visible=False),
            transition=dict(duration=0),
            tickcolor='rgba(0,0,0,0)',
            font=dict(color='rgba(0,0,0,0)', size=1),
            bgcolor='#262626',
            bordercolor='#404040',
            borderwidth=1
        )]
    )

    return fig

# 3. Esecuzione
if __name__ == "__main__":
    csv_file = DATA_DIR / 'nuclear-weapons' / 'nuclear-warhead-stockpiles-lines' / 'nuclear-warhead-stockpiles-lines.csv'
    
    try:
        df, countries = load_and_process_data(csv_file)
        fig = create_interactive_plot(df, countries)
        fig.show()
        print("Visualizzazione aggiornata generata con successo!")
    except FileNotFoundError:
        print(f"Errore: Il file '{csv_file}' non è stato trovato.")

Visualizzazione aggiornata generata con successo!


In [2]:
from const import VISUALIZATIONS_DIR

fig.write_html(VISUALIZATIONS_DIR / "rise-and-fall-of-nuclear-arsenals.html")